# Imports & Settings


In [41]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [178]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

# Splitting

# country_unit

In [116]:
data = combined_data_fully_translated.copy()

In [131]:
def extract_city_country(item):      
    if (type(item) == str):
        item = item.strip()
        # there are some empty (non-nan) values
        if (item == ''):
            return [float('nan'), float('nan')]
    
        item = re.sub(' +', ' ', item) # remove multiple spaces

        if (' ' in item) and ('Eesti' in item):        
            split = item.split(' ')
            return [' '.join(split[1:]), split[0]]
        else: 
            return [float('nan'), item]
    else:
        return [float('nan'), float('nan')]

In [132]:
data['city_municipality'] = data.apply(lambda item: extract_city_country(item['country_and_unit'])[0], axis=1)
data['country'] = data.apply(lambda item: extract_city_country(item['country_and_unit'])[1], axis=1)

In [141]:
data.to_csv('data/prep.csv')

# material

In [165]:
data = prep.copy()

In [159]:
len(data.material.unique())

117

In [166]:
# to make the following work even for nan values

data['material'] = data['material'].replace(np.nan, 'nan')

In [167]:
# prepare single values to be distinguishable
data['material'] = data['material'].apply(lambda x: x.split('>'))

In [171]:
# https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
data = data.join(pd.DataFrame(mlb.fit_transform(data.pop('material')),
                          columns=mlb.classes_,
                          index=data.index))

In [176]:
data.to_csv('data/prep.csv')

# technique

In [179]:
data = prep.copy()

In [180]:
data.head(1)

,full_nr,name,ks,commentary,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,text,class,technique,parameter,unit,value,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,legend,is_original,initial_info,damages,state,color,additional_text,type,source,city_municipality,country,Polish,RC Photo Paper,a pearl,acetate cellulose film,albumen paper,albumin paper,aluminium,amber,artificial fiber material,artificial leather,artificial material,atlas,ballpoint pen ink,birch,bone,brass,brocade (clothing variety),bronze,canvas,canvas (type of cloth),cardboard,cast iron,celluloid,ceramics,chalk,chamois leather,chamotte,charcoal,chromogen emulsion,chromogen paper,clay,clothing variety,collodion paper,colloid paper,colour,copper,cotton,crepe,crepe (cloth type),crystal,diffusion paper,email,emulsion,enamel paint,faience,feather,film,film (material),flint,from the bat,glass,gold,granite,graphite,gypsum,handmade paper,ink,iron,kalka,knitwear,leotard (type of clothing),linen,mascara,metal,metal fibers,moire (clothing variety),movie,nan,newsprint,nitrocellulose film,nut,oil paint,organic matter,paper,papier mache,photo emulsion,photo material,photo paper,photo plate,photographic material,plastic,plastic mass,plywood,porcelain,printing ink,quartz,rubber,salt paper,silk,silver,silver gelatin emulsion,silver gelatin paper,skin,slate,stone,synthetic fibers,synthetic material,tempera,textile,tin,trillion,watercolor paint,wax,white metal,wire,wood,wood material,wooden board,wool,yarn
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
232170,ETMM _ 12150:115 Aj 118:44/M20,"Kuno Areng, Bremerhaven Festwoche medal",118.0,NaN,festivals,linn Bremerhaven,1979,NaN,ei,Saksamaa,participant,"Areng, Kuno",NaN,NaN,NaN,NaN,diameter,cm,4,ETMM,_,12150.0,115.0,NaN,Aj,44.0,M20,1.0,NaN,1.0,Festwoche - Breemenhaven,NaN,good,grey,KUTTER ASTARTE -SCHIFFERGILDE BREMENHAVEN E.V.,medal,train,NaN,Saksamaa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [181]:
data.technique.unique()


array([nan, 'manuscript', 'enamelling', 'black and white photography',
       'handwriting', 'printing', 'copying', 'oil', 'pen', 'watercolor',
       'typing', 'drawing', 'mixed media', 'soft varnish',
       'photography>black and white photography', 'lithography',
       'mascara', 'painting',
       'photography>black and white photography>wet collodion process>pannotype',
       'sketch', 'photomechanical printing>photolithography>collotype',
       'color photography', 'ink', 'copper engraving', 'etching',
       'sewing', 'taking pictures', 'writing',
       'photography>color photography>chromogen procedure', 'gouache',
       'felt tip pen', 'bronzing', 'monotypy', 'digital photography',
       'color photo', 'painting techniques', 'photography',
       'photographic techniques', 'linocut', 'handicraft',
       'steel engraving', 'turning', 'ballpoint pen', 'ink drawing',
       'stamping', '(close/together) sewing', 'black and white photo',
       'pastel', 'toning', 'crochet